In [1]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
from torch import nn
import cv2
from super_image import EdsrModel, ImageLoader
import os

device='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using: {device}')

import warnings
warnings.filterwarnings("ignore")

Using: cuda


In [2]:
def build_generator():
    
    class ResidualBlock(nn.Module):
        def __init__(self, in_channels, out_channels, expansion=6, stride=1, alpha=1.0):
            super(ResidualBlock, self).__init__()
            self.expansion = expansion
            self.stride = stride
            self.in_channels = in_channels
            self.out_channels = int(out_channels * alpha)
            self.pointwise_conv_filters = self._make_divisible(self.out_channels, 8)
            self.conv1 = nn.Conv2d(in_channels, in_channels * expansion, kernel_size=1, stride=1, padding=0, bias=True)
            self.bn1 = nn.BatchNorm2d(in_channels * expansion)
            self.conv2 = nn.Conv2d(in_channels * expansion, in_channels * expansion, kernel_size=3, stride=stride, padding=1, groups=in_channels * expansion, bias=True)
            self.bn2 = nn.BatchNorm2d(in_channels * expansion)
            self.conv3 = nn.Conv2d(in_channels * expansion, self.pointwise_conv_filters, kernel_size=1, stride=1, padding=0, bias=True)
            self.bn3 = nn.BatchNorm2d(self.pointwise_conv_filters)
            self.relu = nn.ReLU(inplace=True)
            self.skip_add = (stride == 1 and in_channels == self.pointwise_conv_filters)

        def forward(self, x):
            identity = x

            out = self.conv1(x)
            out = self.bn1(out)
            out = self.relu(out)

            out = self.conv2(out)
            out = self.bn2(out)
            out = self.relu(out)

            out = self.conv3(out)
            out = self.bn3(out)

            if self.skip_add:
                out = out + identity

            return out

        @staticmethod
        def _make_divisible(v, divisor, min_value=None):
            if min_value is None:
                min_value = divisor
            new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
            if new_v < 0.9 * v:
                new_v += divisor
            return new_v

    class Generator(nn.Module):
        def __init__(self, in_channels, num_residual_blocks, gf):
            super(Generator, self).__init__()
            self.num_residual_blocks = num_residual_blocks
            self.gf = gf

            self.conv1 = nn.Conv2d(in_channels, gf, kernel_size=3, stride=1, padding=1)
            self.bn1 = nn.BatchNorm2d(gf)
            self.prelu1 = nn.PReLU()

            self.residual_blocks = self.make_layer(ResidualBlock, gf, num_residual_blocks)

            self.conv2 = nn.Conv2d(gf, gf, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(gf)

            self.upsample1 = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
                nn.Conv2d(gf, gf, kernel_size=3, stride=1, padding=1),
                nn.PReLU()
            )

            self.upsample2 = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
                nn.Conv2d(gf, gf, kernel_size=3, stride=1, padding=1),
                nn.PReLU()
            )

            self.conv3 = nn.Conv2d(gf, 3, kernel_size=3, stride=1, padding=1)
            self.tanh = nn.Tanh()

        def make_layer(self, block, out_channels, blocks):
            layers = []
            for _ in range(blocks):
                layers.append(block(out_channels, out_channels))
            return nn.Sequential(*layers)

        def forward(self, x):
            out1 = self.prelu1(self.bn1(self.conv1(x)))
            out = self.residual_blocks(out1)
            out = self.bn2(self.conv2(out))
            out = out + out1
            out = self.upsample1(out)
            out = self.upsample2(out)
            out = self.tanh(self.conv3(out))
            return out

    return Generator(3, 6, 32)


In [3]:
class _conv(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, bias):
        super(_conv, self).__init__(in_channels = in_channels, out_channels = out_channels, 
                               kernel_size = kernel_size, stride = stride, padding = (kernel_size) // 2, bias = True)
        
        self.weight.data = torch.normal(torch.zeros((out_channels, in_channels, kernel_size, kernel_size)), 0.02)
        self.bias.data = torch.zeros((out_channels))
        
        for p in self.parameters():
            p.requires_grad = True
        

class conv(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, BN = False, act = None, stride = 1, bias = True):
        super(conv, self).__init__()
        m = []
        m.append(_conv(in_channels = in_channel, out_channels = out_channel, 
                               kernel_size = kernel_size, stride = stride, padding = (kernel_size) // 2, bias = True))
        
        if BN:
            m.append(nn.BatchNorm2d(num_features = out_channel))
        
        if act is not None:
            m.append(act)
        
        self.body = nn.Sequential(*m)
        
    def forward(self, x):
        out = self.body(x)
        return out
        
class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size, act = nn.ReLU(inplace = True), bias = True):
        super(ResBlock, self).__init__()
        m = []
        m.append(conv(channels, channels, kernel_size, BN = True, act = act))
        m.append(conv(channels, channels, kernel_size, BN = True, act = None))
        self.body = nn.Sequential(*m)
        
    def forward(self, x):
        res = self.body(x)
        res += x
        return res
    
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, num_res_block, act = nn.ReLU(inplace = True)):
        super(BasicBlock, self).__init__()
        m = []
        
        self.conv = conv(in_channels, out_channels, kernel_size, BN = False, act = act)
        for i in range(num_res_block):
            m.append(ResBlock(out_channels, kernel_size, act))
        
        m.append(conv(out_channels, out_channels, kernel_size, BN = True, act = None))
        
        self.body = nn.Sequential(*m)
        
    def forward(self, x):
        res = self.conv(x)
        out = self.body(res)
        out += res
        
        return out
        
class Upsampler(nn.Module):
    def __init__(self, channel, kernel_size, scale, act = nn.ReLU(inplace = True)):
        super(Upsampler, self).__init__()
        m = []
        m.append(conv(channel, channel * scale * scale, kernel_size))
        m.append(nn.PixelShuffle(scale))
    
        if act is not None:
            m.append(act)
        
        self.body = nn.Sequential(*m)
    
    def forward(self, x):
        out = self.body(x)
        return out

class discrim_block(nn.Module):
    def __init__(self, in_feats, out_feats, kernel_size, act = nn.LeakyReLU(inplace = True)):
        super(discrim_block, self).__init__()
        m = []
        m.append(conv(in_feats, out_feats, kernel_size, BN = True, act = act))
        m.append(conv(out_feats, out_feats, kernel_size, BN = True, act = act, stride = 2))
        self.body = nn.Sequential(*m)
        
    def forward(self, x):
        out = self.body(x)
        return out

class MiniSRGAN(nn.Module):
    
    def __init__(self, img_feat = 3, n_feats = 64, kernel_size = 3, num_block = 8, act = nn.PReLU(), scale=4):
        super(MiniSRGAN, self).__init__()
        
        self.conv01 = conv(in_channel = img_feat, out_channel = n_feats, kernel_size = 9, BN = False, act = act)
        
        resblocks = [ResBlock(channels = n_feats, kernel_size = 3, act = act) for _ in range(num_block)]
        self.body = nn.Sequential(*resblocks)
        
        self.conv02 = conv(in_channel = n_feats, out_channel = n_feats, kernel_size = 3, BN = True, act = None)
        
        if(scale == 4):
            upsample_blocks = [Upsampler(channel = n_feats, kernel_size = 3, scale = 2, act = act) for _ in range(2)]
        else:
            upsample_blocks = [Upsampler(channel = n_feats, kernel_size = 3, scale = scale, act = act)]

        self.tail = nn.Sequential(*upsample_blocks)
        
        self.last_conv = conv(in_channel = n_feats, out_channel = img_feat, kernel_size = 3, BN = False, act = nn.Tanh())
        
    def forward(self, x):
        
        x = self.conv01(x)
        _skip_connection = x
        
        x = self.body(x)
        x = self.conv02(x)
        feat = x + _skip_connection
        
        x = self.tail(feat)
        x = self.last_conv(x)
        
        return x, feat



#### FSRCNN (https://github.com/Lornatang/FSRCNN-PyTorch/tree/master)

In [4]:
import math
from torch import nn


class FSRCNN(nn.Module):
    def __init__(self, scale_factor, num_channels=1, d=56, s=12, m=4):
        super(FSRCNN, self).__init__()
        self.first_part = nn.Sequential(
            nn.Conv2d(num_channels, d, kernel_size=5, padding=5//2),
            nn.PReLU(d)
        )
        self.mid_part = [nn.Conv2d(d, s, kernel_size=1), nn.PReLU(s)]
        for _ in range(m):
            self.mid_part.extend([nn.Conv2d(s, s, kernel_size=3, padding=3//2), nn.PReLU(s)])
        self.mid_part.extend([nn.Conv2d(s, d, kernel_size=1), nn.PReLU(d)])
        self.mid_part = nn.Sequential(*self.mid_part)
        self.last_part = nn.ConvTranspose2d(d, num_channels, kernel_size=9, stride=scale_factor, padding=9//2,
                                            output_padding=scale_factor-1)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.first_part:
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data, mean=0.0, std=math.sqrt(2/(m.out_channels*m.weight.data[0][0].numel())))
                nn.init.zeros_(m.bias.data)
        for m in self.mid_part:
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data, mean=0.0, std=math.sqrt(2/(m.out_channels*m.weight.data[0][0].numel())))
                nn.init.zeros_(m.bias.data)
        nn.init.normal_(self.last_part.weight.data, mean=0.0, std=0.001)
        nn.init.zeros_(self.last_part.bias.data)

    def forward(self, x):
        x = self.first_part(x)
        x = self.mid_part(x)
        x = self.last_part(x)
        return x

import torch
import numpy as np


def calc_patch_size(func):
    def wrapper(args):
        if args.scale == 2:
            args.patch_size = 10
        elif args.scale == 3:
            args.patch_size = 7
        elif args.scale == 4:
            args.patch_size = 6
        else:
            raise Exception('Scale Error', args.scale)
        return func(args)
    return wrapper


def convert_rgb_to_y(img, dim_order='hwc'):
    if dim_order == 'hwc':
        return 16. + (64.738 * img[..., 0] + 129.057 * img[..., 1] + 25.064 * img[..., 2]) / 256.
    else:
        return 16. + (64.738 * img[0] + 129.057 * img[1] + 25.064 * img[2]) / 256.


def convert_rgb_to_ycbcr(img, dim_order='hwc'):
    if dim_order == 'hwc':
        y = 16. + (64.738 * img[..., 0] + 129.057 * img[..., 1] + 25.064 * img[..., 2]) / 256.
        cb = 128. + (-37.945 * img[..., 0] - 74.494 * img[..., 1] + 112.439 * img[..., 2]) / 256.
        cr = 128. + (112.439 * img[..., 0] - 94.154 * img[..., 1] - 18.285 * img[..., 2]) / 256.
    else:
        y = 16. + (64.738 * img[0] + 129.057 * img[1] + 25.064 * img[2]) / 256.
        cb = 128. + (-37.945 * img[0] - 74.494 * img[1] + 112.439 * img[2]) / 256.
        cr = 128. + (112.439 * img[0] - 94.154 * img[1] - 18.285 * img[2]) / 256.
    return np.array([y, cb, cr]).transpose([1, 2, 0])


def convert_ycbcr_to_rgb(img, dim_order='hwc'):
    if dim_order == 'hwc':
        r = 298.082 * img[..., 0] / 256. + 408.583 * img[..., 2] / 256. - 222.921
        g = 298.082 * img[..., 0] / 256. - 100.291 * img[..., 1] / 256. - 208.120 * img[..., 2] / 256. + 135.576
        b = 298.082 * img[..., 0] / 256. + 516.412 * img[..., 1] / 256. - 276.836
    else:
        r = 298.082 * img[0] / 256. + 408.583 * img[2] / 256. - 222.921
        g = 298.082 * img[0] / 256. - 100.291 * img[1] / 256. - 208.120 * img[2] / 256. + 135.576
        b = 298.082 * img[0] / 256. + 516.412 * img[1] / 256. - 276.836
    return np.array([r, g, b]).transpose([1, 2, 0])


def preprocess(img, device):
    img = np.array(img).astype(np.float32)
    ycbcr = convert_rgb_to_ycbcr(img)
    x = ycbcr[..., 0]
    x /= 255.
    x = torch.from_numpy(x).to(device)
    x = x.unsqueeze(0).unsqueeze(0)
    return x, ycbcr


def calc_psnr(img1, img2):
    return 10. * torch.log10(1. / torch.mean((img1 - img2) ** 2))


class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count




In [5]:
class TinySRGAN(nn.Module):
    
    def __init__(self, img_feat = 3, n_feats = 32, kernel_size = 3, num_block = 6, act = nn.PReLU(), scale=4):
        super(TinySRGAN, self).__init__()
        
        self.conv01 = conv(in_channel = img_feat, out_channel = n_feats, kernel_size = 9, BN = False, act = act)
        
        resblocks = [ResBlock(channels = n_feats, kernel_size = 3, act = act) for _ in range(num_block)]
        self.body = nn.Sequential(*resblocks)
        
        self.conv02 = conv(in_channel = n_feats, out_channel = n_feats, kernel_size = 3, BN = True, act = None)
        
        if(scale == 4):
            upsample_blocks = [Upsampler(channel = n_feats, kernel_size = 3, scale = 2, act = act) for _ in range(2)]
        else:
            upsample_blocks = [Upsampler(channel = n_feats, kernel_size = 3, scale = scale, act = act)]

        self.tail = nn.Sequential(*upsample_blocks)
        
        self.last_conv = conv(in_channel = n_feats, out_channel = img_feat, kernel_size = 3, BN = False, act = nn.Tanh())
        
    def forward(self, x):
        
        x = self.conv01(x)
        _skip_connection = x
        
        x = self.body(x)
        x = self.conv02(x)
        feat = x + _skip_connection
        
        x = self.tail(feat)
        x = self.last_conv(x)
        
        return x, feat



In [6]:
class SRGAN(nn.Module):
    
    def __init__(self, img_feat = 3, n_feats = 64, kernel_size = 3, num_block = 16, act = nn.PReLU(), scale=4):
        super(SRGAN, self).__init__()
        
        self.conv01 = conv(in_channel = img_feat, out_channel = n_feats, kernel_size = 9, BN = False, act = act)
        
        resblocks = [ResBlock(channels = n_feats, kernel_size = 3, act = act) for _ in range(num_block)]
        self.body = nn.Sequential(*resblocks)
        
        self.conv02 = conv(in_channel = n_feats, out_channel = n_feats, kernel_size = 3, BN = True, act = None)
        
        if(scale == 4):
            upsample_blocks = [Upsampler(channel = n_feats, kernel_size = 3, scale = 2, act = act) for _ in range(2)]
        else:
            upsample_blocks = [Upsampler(channel = n_feats, kernel_size = 3, scale = scale, act = act)]

        self.tail = nn.Sequential(*upsample_blocks)
        
        self.last_conv = conv(in_channel = n_feats, out_channel = img_feat, kernel_size = 3, BN = False, act = nn.Tanh())
        
    def forward(self, x):
        
        x = self.conv01(x)
        _skip_connection = x
        
        x = self.body(x)
        x = self.conv02(x)
        feat = x + _skip_connection
        
        x = self.tail(feat)
        x = self.last_conv(x)
        
        return x, feat



In [7]:
def numpify(imgs):
    all_images = []
    for img in imgs:
        img = img.permute(1,2,0).to('cpu') ### MIGHT CRASH HERE
        all_images.append(img)
    return np.stack(all_images, axis=0)

transform = transforms.Compose([
            transforms.ToTensor()
        ])

def translate_image(image, sharpen, model_name, save):
    print('Translating!')


    if(model_name=='MobileSR'):
        
        model=build_generator().to(device)
        model.load_state_dict(torch.load('../weights/mobile_sr.pt'))

        low_res = transform(image)
        low_res = low_res.unsqueeze(dim=0).to(device)
        model.eval()
        with torch.no_grad():
            sr = model(low_res)
            
        fake_imgs = numpify(sr)
        
        sr_img = Image.fromarray((((fake_imgs[0] + 1) / 2) * 255).astype(np.uint8))

    elif(model_name=='EDSR'):
        model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=4)    
        inputs = ImageLoader.load_image(image)
        with torch.no_grad():
            preds = model(inputs)

        sr_img = preds.data.cpu().numpy()
        sr_img = sr_img[0].transpose((1, 2, 0)) * 255.0
        sr_img = Image.fromarray(sr_img.astype(np.uint8))
    elif(model_name=='MiniSRGAN'):
        model = MiniSRGAN().to(device)
        model.load_state_dict(torch.load('../weights/miniSRGAN.pt'))
        
        inputs = np.array(image)
        inputs = (inputs / 127.5) - 1.0   
        inputs = torch.tensor(inputs.transpose(2, 0, 1).astype(np.float32)).to(device)
        model.eval()
        
        with torch.no_grad():
            output, _ = model(torch.unsqueeze(inputs,dim=0))
        output = output[0].cpu().numpy()
        output = (output + 1.0) / 2.0
        output = output.transpose(1, 2, 0)
        sr_img = Image.fromarray((output * 255.0).astype(np.uint8))
    elif(model_name=='miniSRResNET'):
        model = MiniSRGAN().to(device)
        model.load_state_dict(torch.load('../weights/miniSRResNET.pt'))
        
        inputs = np.array(image)
        inputs = (inputs / 127.5) - 1.0   
        inputs = torch.tensor(inputs.transpose(2, 0, 1).astype(np.float32)).to(device)
        model.eval()
        
        with torch.no_grad():
            output, _ = model(torch.unsqueeze(inputs,dim=0))
        output = output[0].cpu().numpy()
        output = (output + 1.0) / 2.0
        output = output.transpose(1, 2, 0)
        sr_img = Image.fromarray((output * 255.0).astype(np.uint8))
    elif(model_name=='TinySRGAN'):
        model = TinySRGAN().to(device)
        model.load_state_dict(torch.load('../weights/tinySRGAN.pt'))
        
        inputs = np.array(image)
        inputs = (inputs / 127.5) - 1.0   
        inputs = torch.tensor(inputs.transpose(2, 0, 1).astype(np.float32)).to(device)
        model.eval()
        
        with torch.no_grad():
            output, _ = model(torch.unsqueeze(inputs,dim=0))
        output = output[0].cpu().numpy()
        output = (output + 1.0) / 2.0
        output = output.transpose(1, 2, 0)
        sr_img = Image.fromarray((output * 255.0).astype(np.uint8))
    elif(model_name=='SRGAN'):
        model = SRGAN().to(device)
        model.load_state_dict(torch.load('../weights/SRGAN.pt'))
        
        inputs = np.array(image)
        inputs = (inputs / 127.5) - 1.0   
        inputs = torch.tensor(inputs.transpose(2, 0, 1).astype(np.float32)).to(device)
        model.eval()
        
        with torch.no_grad():
            output, _ = model(torch.unsqueeze(inputs,dim=0))
        output = output[0].cpu().numpy()
        output = (output + 1.0) / 2.0
        output = output.transpose(1, 2, 0)
        sr_img = Image.fromarray((output * 255.0).astype(np.uint8))
    
    if sharpen:
        sr_img_cv = np.array(sr_img)
        sr_img_cv = cv2.cvtColor(sr_img_cv, cv2.COLOR_RGB2BGR)
        
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_sr_img_cv = cv2.filter2D(sr_img_cv, -1, kernel)
        
        sharpened_sr_img = Image.fromarray(cv2.cvtColor(sharpened_sr_img_cv, cv2.COLOR_BGR2RGB))

        if(save=="True"):
            sharpened_sr_img.save('super_resolved_image.png')
        
        return sharpened_sr_img
    else:
        
        if(save=="True"):
            sr_img.save('super_resolved_image.png')
        
        return sr_img


In [8]:
degraded_images = os.listdir('./Set5/image_SRF_4/LR')
for image_name in degraded_images:
    print(f"Processing {image_name}...")
    image_path=os.path.join('./Set5/image_SRF_4/LR', image_name)
    image = Image.open(image_path)
    sr_img = translate_image(image, sharpen=False, model_name='MobileSR', save='False')
    if(os.path.exists('./super_resolved_Set5_images_mobileSR')==False):
        os.mkdir('./super_resolved_Set5_images_mobileSR')
    else:
        sr_img.save(f'./super_resolved_Set5_images_mobileSR/{image_name}')

Processing 3.png...
Translating!
Processing 2.png...
Translating!
Processing 5.png...
Translating!
Processing 1.png...
Translating!
Processing 4.png...
Translating!


In [9]:
degraded_images = os.listdir('./Set5/image_SRF_4/LR')
for image_name in degraded_images:
    print(f"Processing {image_name}...")
    image_path=os.path.join('./Set5/image_SRF_4/LR', image_name)
    image = Image.open(image_path)
    sr_img = translate_image(image, sharpen=False, model_name='MiniSRGAN', save='False')
    if(os.path.exists('./super_resolved_Set5_images_miniSRGAN')==False):
        os.mkdir('./super_resolved_Set5_images_miniSRGAN')
    else:
        sr_img.save(f'./super_resolved_Set5_images_miniSRGAN/{image_name}')

Processing 3.png...
Translating!
Processing 2.png...
Translating!
Processing 5.png...
Translating!
Processing 1.png...
Translating!
Processing 4.png...
Translating!


In [10]:
degraded_images = os.listdir('./Set5/image_SRF_4/LR')
for image_name in degraded_images:
    print(f"Processing {image_name}...")
    image_path=os.path.join('./Set5/image_SRF_4/LR', image_name)
    image = Image.open(image_path)
    sr_img = translate_image(image, sharpen=False, model_name='EDSR', save='False')
    if(os.path.exists('./super_resolved_Set5_images_EDSR')==False):
        os.mkdir('./super_resolved_Set5_images_EDSR')
    else:
        sr_img.save(f'./super_resolved_Set5_images_EDSR/{image_name}')

Processing 3.png...
Translating!
https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_4x.pt
Processing 2.png...
Translating!
https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_4x.pt
Processing 5.png...
Translating!
https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_4x.pt
Processing 1.png...
Translating!
https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_4x.pt
Processing 4.png...
Translating!
https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_4x.pt


In [11]:
degraded_images = os.listdir('../../../../Set5/image_SRF_4/LR')
for image_name in degraded_images:
    print(f"Processing {image_name}...")
    image_path=os.path.join('../../../../Set5/image_SRF_4/LR', image_name)
    image = Image.open(image_path)
    sr_img = translate_image(image, sharpen=False, model_name='miniSRResNET', save='False')
    if(os.path.exists('./super_resolved_Set5_images_miniSRResNET')==False):
        os.mkdir('./super_resolved_Set5_images_miniSRResNET')
    else:
        sr_img.save(f'./super_resolved_Set5_images_miniSRResNET/{image_name}')

Processing 3.png...
Translating!
Processing 2.png...
Translating!
Processing 5.png...
Translating!
Processing 1.png...
Translating!
Processing 4.png...
Translating!


In [12]:
degraded_images = os.listdir('../../../../Set5/image_SRF_4/LR')
for image_name in degraded_images:
    print(f"Processing {image_name}...")
    image_path=os.path.join('../../../../Set5/image_SRF_4/LR', image_name)
    image = Image.open(image_path)
    sr_img = translate_image(image, sharpen=False, model_name='TinySRGAN', save='False')
    if(os.path.exists('./super_resolved_Set5_images_tinySRGAN')==False):
        os.mkdir('./super_resolved_Set5_images_tinySRGAN')
    else:
        sr_img.save(f'./super_resolved_Set5_images_tinySRGAN/{image_name}')

Processing 3.png...
Translating!
Processing 2.png...
Translating!
Processing 5.png...
Translating!
Processing 1.png...
Translating!
Processing 4.png...
Translating!


In [13]:
degraded_images = os.listdir('../../../../Set5/image_SRF_4/LR')
for image_name in degraded_images:
    print(f"Processing {image_name}...")
    image_path=os.path.join('../../../../Set5/image_SRF_4/LR', image_name)
    image = Image.open(image_path)
    sr_img = translate_image(image, sharpen=False, model_name='SRGAN', save='False')
    if(os.path.exists('./super_resolved_Set5_images_srgan')==False):
        os.mkdir('./super_resolved_Set5_images_srgan')
    else:
        sr_img.save(f'./super_resolved_Set5_images_srgan/{image_name}')

Processing 3.png...
Translating!
Processing 2.png...
Translating!
Processing 5.png...
Translating!
Processing 1.png...
Translating!
Processing 4.png...
Translating!


In [14]:
import PIL.Image as pil_image
import os 

model=FSRCNN(scale_factor=4).to(device)
state_dict = model.state_dict()
for n, p in torch.load('../weights/fsrcnn_x4.pth', map_location=lambda storage, loc: storage).items():
    if n in state_dict.keys():
        state_dict[n].copy_(p)
    else:
        raise KeyError(n)

model.eval()

scale=4

degraded_images = os.listdir('../../../../Set5/image_SRF_4/HR')
for image_name in degraded_images:
    print(f"Processing {image_name}...")
    image_path=os.path.join('../../../../Set5/image_SRF_4/HR', image_name)
    image = pil_image.open(image_path)
    
    image_width = (image.width // scale) * scale
    image_height = (image.height // scale) * scale
    
    hr = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
    lr = hr.resize((hr.width // scale, hr.height // scale), resample=pil_image.BICUBIC)
    bicubic = lr.resize((lr.width * scale, lr.height * scale), resample=pil_image.BICUBIC)
    #bicubic.save(image_file.replace('.', '_bicubic_x{}.'.format(scale)))

    
    lr, _ = preprocess(lr, device)
    hr, _ = preprocess(hr, device)
    _, ycbcr = preprocess(bicubic, device)

    with torch.no_grad():
        preds = model(lr).clamp(0.0, 1.0)

    psnr = calc_psnr(hr, preds)
    print('PSNR: {:.2f}'.format(psnr))

    preds = preds.mul(255.0).cpu().numpy().squeeze(0).squeeze(0)

    output = np.array([preds, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])
    output = np.clip(convert_ycbcr_to_rgb(output), 0.0, 255.0).astype(np.uint8)
    sr_img = pil_image.fromarray(output)

    #sr_img = translate_image(image, sharpen=False, model_name='miniSRResNET', save='False')
    
    
    if(os.path.exists('./super_resolved_Set5_images_FSRCNN')==False):
        os.mkdir('./super_resolved_Set5_images_FSRCNN')
    else:
        sr_img.save(f'./super_resolved_Set5_images_FSRCNN/{image_name}')

Processing 3.png...
PSNR: 25.33
Processing 2.png...
PSNR: 32.46
Processing 5.png...
PSNR: 29.11
Processing 1.png...
PSNR: 33.14
Processing 4.png...
PSNR: 32.47
